*By DJun (github.com/djun); 2021-8-16*

#### 作业题：

抓取网易新闻首页中的多条新闻，仅抓取那些 每一块的左边是标题、XXX时间、XXX跟帖 右边附带一张封面图的 那些新闻；

抓取每篇文章的标题、链接、发布日期、媒体名称、跟帖数量、正文内容（仅文字内容 不包含图片视频等）、封面图片；

作业结果要求：
- 新闻存放的格式是，以新闻标题起名文件夹，每个文件夹中包含一个文本文件记录这篇文章上述的文本内容，以及封面图片的文件
- 在这些新闻标题文件夹的外面，生成一个汇总 标题、链接、发布日期、媒体名称、跟帖数量 的csv表格文件

提供抓取的首页链接（还是那个）：https://3g.163.com/touch/news

新闻-数码分区：https://3g.163.com/touch/digi/?ver=c&clickfrom=index2018_header_main

首页新闻项的参考XPath：
`//div[contains(@class, 'tab-content')]//article`

In [ ]:
# 这篇作业题是从写给实习生的爬虫入门资料中抽取出来的，现在来做一个开发过程的全面演示哈

# 先看一下原来资料的内容

# 这次挑选了数码分区的新闻来做，其实页面结构是一样的（跟上次给的链接对比）

# 需要先理解一下作业题的题面哈

# 在页面那边分析发现，符合“每一块的左边是标题、XXX时间、XXX跟帖 右边附带一张封面图的”要求的新闻，是article标签名、class包含card-type-news的那些标签

# 利用好参考资料中的例子，可以快速地把每一部分写出来

# 先解决获取页面的部分

# 获取首页、整理首页上的大致的文章信息已经完成了，已经获取“标题、链接、封面图片”

# 接下来是进入每篇文章的正文，获取“发布日期、媒体名称、跟帖数量、正文内容（仅文字内容 不包含图片视频等）”

# 先按要求写好“建立文件夹”的部分

In [1]:
import requests

URL = "https://3g.163.com/touch/digi/?ver=c&clickfrom=index2018_header_main"  # 避免反复复制粘贴

r = requests.get(URL)

# 设置文本编码
r.encoding = "utf-8"

# 查看解码后的文本内容（就是HTML代码）
r.text

'<!DOCTYPE html>\n<html lang="en">\n  <head>\n    <meta charset="UTF-8" />\n    <link rel="dns-prefetch" href="//cms-bucket.nosdn.127.net" />\n    <link rel="dns-prefetch" href="//pic-bucket.nosdn.127.net" />\n    <link rel="dns-prefetch" href="//static.ws.126.net" />\n    <meta name="viewport" content="width=device-width, initial-scale=1.0" />\n    <meta http-equiv="X-UA-Compatible" content="ie=edge" />\n    <title>网易数码_手机网易网</title>\n    <meta content="telephone=no" name="format-detection" />\n    <meta name="keywords" content="网易数码 笔记本电脑,平板电脑,数码相机DC,苹果电脑图片,报价,论坛" />\n    <meta name="description" content="网易数码 笔记本电脑,平板电脑,数码相机DC,苹果电脑图片、报价、论坛。手机网易网数码频道-https://3g.163.com/touch/digi" />\n    <meta name="google-site-verification" content="vDJDt0eLizo98mTqFjhG4ONEm8DlFI7bdonVyRDi-EY" />\n    <script type=\'text/javascript\'>window._ANT_PROJECT_ID="NTM-5AE0KFYY-2";(function(){var domainMatches=[{test:/163\\.com$/i,domain:"163"},{test:/baidu\\.com$/i,domain:"baidu"},{test:/sm\\.cn$/i,domain

In [2]:
from lxml import html

# 将HTML代码转换为HtmlElement（树节点对象）
# 目前拿到的tree对象，其实就是根节点的对象
# 查找我们所需的信息，一般也是从根节点开始查找
tree = html.fromstring(r.text)
type(tree)

lxml.html.HtmlElement

In [4]:
# 提取元素对象列表
articles = tree.xpath("//div[contains(@class, 'tab-content')]//article[contains(@class, 'card-type-news')]")
# 先用text()取到文字看看
# articles = tree.xpath("//div[contains(@class, 'tab-content')]//article[contains(@class, 'card-type-news')]//h3/text()")
articles

[<Element article at 0x22a051b1818>,
 <Element article at 0x22a051b19f8>,
 <Element article at 0x22a051b1958>,
 <Element article at 0x22a051b1a48>,
 <Element article at 0x22a051b1a98>,
 <Element article at 0x22a051b1ae8>,
 <Element article at 0x22a051b1b38>,
 <Element article at 0x22a051b1b88>,
 <Element article at 0x22a051b1bd8>,
 <Element article at 0x22a051b1c28>,
 <Element article at 0x22a051b1c78>,
 <Element article at 0x22a051b1cc8>,
 <Element article at 0x22a051b1d18>,
 <Element article at 0x22a051b1d68>,
 <Element article at 0x22a051b1db8>,
 <Element article at 0x22a051b1e08>,
 <Element article at 0x22a051b1e58>,
 <Element article at 0x22a051b1ea8>,
 <Element article at 0x22a051b1ef8>,
 <Element article at 0x22a051b1f48>,
 <Element article at 0x22a051b1f98>,
 <Element article at 0x22a051b7048>,
 <Element article at 0x22a051b7098>,
 <Element article at 0x22a051b70e8>,
 <Element article at 0x22a051b7138>,
 <Element article at 0x22a051b7188>,
 <Element article at 0x22a051b71d8>,
 

In [23]:
from urllib.parse import urljoin, urlparse

# 整理文章信息
# 仅仅是xpath提取后整理信息，生成 list-dict的数据形式
new_articles = []
for a in articles:
    title = a.xpath(".//h3/text()")[0]
    replies = a.xpath(".//span[contains(@class, 'reply')]/text()")[0]  # 跟帖数改在这里取哈
    href = urljoin(URL, a.xpath(".//a/@href")[0])  # 拼合url
    pic_src = urljoin(URL, a.xpath(".//img/@src")[0])  # 拼合url
    
    new_articles.append({
        "title": title,
        "href": href,
        "pic_src": pic_src,
        "replies": replies,
    })

new_articles

[{'title': '罗永浩吐槽新版Apple Music难用：省去海量脏话',
  'href': 'https://3g.163.com/digi/article/GHGUH47S001697V8.html?clickfrom=channel2018_digi_index_newslist#child=index&offset=118',
  'pic_src': 'https://cms-bucket.ws.126.net/2021/0816/f5ddcf83p00qxwsva0016c0009c0070c.png?imageView&thumbnail=234y146&quality=45&interlace=1&enlarge=1&type=png',
  'replies': '1093跟贴'},
 {'title': '微软承诺对Win11会对更多的内置应用程序进行改造',
  'href': 'https://3g.163.com/digi/article/GHGUC2T6001697V8.html?clickfrom=channel2018_digi_index_newslist#child=index&offset=119',
  'pic_src': 'https://cms-bucket.ws.126.net/2021/0816/37703693p00qxwsqp000qc0009c0070c.png?imageView&thumbnail=234y146&quality=45&interlace=1&enlarge=1&type=png',
  'replies': '0跟贴'},
 {'title': '复刻镜头玩上瘾 福伦达将推50mmF1.5镜头',
  'href': 'https://3g.163.com/digi/article/GHGTV897001697V8.html?clickfrom=channel2018_digi_index_newslist#child=index&offset=120',
  'pic_src': 'https://cms-bucket.ws.126.net/2021/0816/846111b0p00qxwsf0001cc0009c0070c.png?imageView&thumbnail=23

In [24]:
import os
from time import sleep

# 做每篇文章具体的事情
for a in new_articles:
    # 准备输出目录
    # 以新闻标题起名文件夹
    output_dir = a["title"]
    # 注意“窄边框+3:2屏幕比例”中的冒号不能作为文件名、文件夹名的问题！
    output_dir = output_dir.replace(":", "：")  # 简单将英文冒号替换成中文冒号
    a["output_dir"] = output_dir  # 把它保存起来，待会写入具体文件时拼接路径会用到
    os.makedirs(output_dir, exist_ok=True)
    
    # 获取正文了接下来
    href = a["href"]
    r = requests.get(href)
    r.encoding = "utf-8"
    tree = html.fromstring(r.text)
    content = tree.xpath("string(//div[@class='article-content']/div[@class='content'])")  # 这里简单粗暴把所有段落直接全部转换成文字
    pub_time = tree.xpath("//span[contains(@class, 'js-time')]/text()")
    pub_time = pub_time[0] if len(pub_time) > 0 else ""
    source = tree.xpath("//a[contains(@class, 'source-sub')]/text()")
    source = source[0] if len(source) > 0 else ""
    
    a.update({
        "content": content,
        "pub_time": pub_time,
        "source": source,
    })

    sleep(0.01)  # 请求不要太频繁

os.listdir(".")

['.ipynb_checkpoints',
 '128GB显存史无前例 AMD确认CDNA2架构加速卡年底亮相',
 '20210813__爬虫入门1--参考.ipynb',
 '20210816__爬虫入门1--作业演示1.ipynb',
 '20210816__爬虫入门1--作业演示模板.ipynb',
 'AMD CDNA2架构年底问世：128GB显存史无前例',
 'AMD下代显卡曝光：Navi 31、Navi 33已获确认',
 'iOS开始审查用户照片：为“后门”问题统一员工口径',
 'LGD都不这么玩，小米OLED电视为啥这么便宜？',
 'realme Book正面亮屏照曝光：窄边框+3：2屏幕比例',
 'TWS耳机销量5年暴增25倍：电池工厂忙扩产',
 'Windows 365云电脑存漏洞被攻破，被黑客成功窃取',
 '下一代Fitbit Charge 5手环泄露：现在有了彩色屏幕',
 '为何做仿生四足机器人？小米常程发文答疑解惑',
 '二手真香警告？ 苹果官方翻新产品悄然上新',
 '以CSAM之名扫描手机，苹果提醒员工为iPhone后门做好准备',
 '侵犯4G LTE专利？苹果将被勒令赔偿3亿美元',
 '华为鸿蒙这个独占功能，所有安卓手机都能用了',
 '国产屏下摄像头方案领先三星！王腾：部分指标反超',
 '复刻镜头玩上瘾 福伦达将推50mmF1.5镜头',
 '夫妻俩火车上手机外放刷视频 结果多人被围殴',
 '实机曝光！天美联手Xbox打造战略手游《重返帝国》',
 '富士秋季品发布会将定于9月初：新机新镜来了',
 '微软Instagram账号发梗图嘲笑自家Xbox Series X的命名法',
 '微软承诺对Win11会对更多的内置应用程序进行改造',
 '微软新款Surface笔记本曝光：采用全新设计语言',
 '性能彪悍！苹果重磅新品开产：搭M1升级版',
 '挑战HomePod mini ？499元小米Sound究竟有何能耐',
 '提前体验新系统！网页版Win11上线：不用下载安装',
 '消失4年后，苹果曾经最失败产品要卷土重来了',
 '男子乘坐滴滴被收4元空调费 司机直接甩锅滴滴官方',
 '科技满满的微型摄像头有多可怕？看完细思极恐',
 '罗永浩吐槽新版Apple Music难用：省去海量脏话',
 

In [25]:
new_articles[0]

{'title': '罗永浩吐槽新版Apple Music难用：省去海量脏话',
 'href': 'https://3g.163.com/digi/article/GHGUH47S001697V8.html?clickfrom=channel2018_digi_index_newslist#child=index&offset=118',
 'pic_src': 'https://cms-bucket.ws.126.net/2021/0816/f5ddcf83p00qxwsva0016c0009c0070c.png?imageView&thumbnail=234y146&quality=45&interlace=1&enlarge=1&type=png',
 'replies': '1093跟贴',
 'output_dir': '罗永浩吐槽新版Apple Music难用：省去海量脏话',
 'content': '近日，@罗永浩 发布微博吐槽苹果新版Apple Music，称新版Apple Music虽然音质不错，但是功能难用到令自己这个优秀的产品经理看不下去，而且老罗还是用了“此处省去海量脏话”来形容新版Apple Music的难用。\n                  \n                      \n                      \n                  \n              该微博原文内容如下：“因为spotify迟迟没有上无损，而apple music已经上了，所以忍不住在 Mac 上试了一下。音质还好，但这xx软件的新版难用到我就...此处省去海量脏话。作为这个星球上最优秀的软件产品经理（嗯，之一，如果算上过世的），对我来说，输赢成败发财破产之类的早就不是事了，但看着xx软件胜出，确实有点失眠。”\n                  \n                      \n                      \n                  \n              苹果于不久前推出新版Apple Music，针对其订阅用户，评估在新版Apple Music中带来了无损（Lossless）音乐和杜比全景声（Dolby Atmos）服务，并且维持原订阅价格不变。

In [26]:
# 做每篇文章具体的事情 #2
for a in new_articles:
    # 下载图片
    pic_src = a["pic_src"]
    output_dir = a["output_dir"]
    r = requests.get(pic_src)  # 还是通过GET方法，获取图片数据
    filename = os.path.basename(urlparse(pic_src).path)  # 解析出链接中的文件名
    
    # 拼接路径，写到指定输出目录下
    with open(os.path.join(output_dir, filename), "wb") as fp:
        fp.write(r.content)  # 与r.text的解码文本不同，r.content是原始的未解码数据
    
    sleep(0.01)  # 请求不要太频繁

In [27]:
# 做每篇文章具体的事情 #3
for a in new_articles:
    # 写入正文
    content = a["content"]
    output_dir = a["output_dir"]
    
    # 拼接路径，写到指定输出目录下
    filename = output_dir + ".txt"
    with open(os.path.join(output_dir, filename), "w", encoding="utf-8") as fp:
        fp.write(content)

In [28]:
# 做一张汇总表

from csv import DictWriter

fieldnames = ['标题', '链接', '发布日期', '媒体名称', '跟帖数量']

with open("news.csv", "w", encoding="utf-8", newline='') as fp:  # 记得要加newline=''；官方文档中有说明
    w = DictWriter(fp, fieldnames=fieldnames)
    w.writeheader()  # 写入表头
    for a in new_articles:
        row = {
            '标题': a['title'],
            '链接': a['href'],
            '发布日期': a['pub_time'],
            '媒体名称': a['source'],
            '跟帖数量': a['replies'],
        }
        w.writerow(row)

# 执行完成后，用EmEditor、UltraEdit等软件去查看news.csv

# 全部完成啦！谢谢观看！